In [1]:
import numpy as np
import pandas as pd
import os
import unicodedata

In [2]:
data_dir = '../data/'
output_dir = '../output/'

## 回答者の年収構造と解答を取得する

In [3]:
df_answer = pd.read_excel(os.path.join(data_dir, '240306_受診判断調査_ローデータ.xlsx'), sheet_name='吸込', header=0)
df_answer = df_answer.rename(columns={'SAMPLENUMBER': 'id'})

In [4]:
df_answer

,id,SC1,SC2,SC3,SC4-1,SC4-2,SC4-3,SC4-4,SC4-5,SC4-6,...,Q502,QX1,QX2,JOB,MAR,INC,CHI,BUS,DISP,WARI
0,I/10,S/2,I/10,S/47,M/19,NaN,NaN,NaN,NaN,NaN,...,S/4,S/4,S/3,S/11,S/2,S/10,S/2,S/43,S/6,S/22
1,NaN,SC1 性別,SC2 年齢,SC3 居住地,SC4 子どもの年齢,NaN,NaN,NaN,NaN,NaN,...,Q502 ChatGPT回答に対する受診判断／子供に頭のけが・首のけが。受傷後、すでに2 日何事,QX1 ChatGPT回答に対する信頼度,QX2 ChatGPT使用による精神状態の変化,職業,未既婚,世帯年収,子供の有無,業種,DISP 回答した調査画面,WARI 設定された割付
2,NaN,NaN,NaN,NaN,0歳,1歳,2歳,3歳,4歳,5歳,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,40,14,0,0,0,0,0,0,...,NaN,3,3,3,2,3,2,19,1,6
4,2,1,52,27,0,0,0,0,0,0,...,NaN,4,2,5,2,5,1,13,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,195,1,44,1,0,0,0,0,0,0,...,NaN,2,2,1,2,3,1,41,6,18
156,196,1,49,12,0,0,0,0,0,0,...,NaN,4,3,5,2,10,1,30,6,15
157,197,2,42,25,0,0,0,0,0,0,...,NaN,2,1,4,2,7,1,10,6,11
158,198,1,49,11,0,0,0,0,0,0,...,NaN,2,3,2,2,5,1,36,6,19


In [5]:
ids = np.array(df_answer.loc[3:159, 'id'].astype(int))

In [6]:
measured_incomes = np.array(df_answer.loc[3:159, 'INC'].astype(int))

In [7]:
df_income = pd.DataFrame({
    'id': ids,
    'income': measured_incomes
})

In [8]:
df_income_annotate = pd.read_excel(os.path.join(data_dir, '240306_受診判断調査_項目見出し.xlsx'), sheet_name='item', header=0)

In [9]:
def clean_convert(text):
    return unicodedata.normalize('NFKC', text)

In [10]:
set_range = np.array(df_income_annotate.iloc[2610:2620, 2])
set_range = np.array([clean_convert(income) for income in set_range])

In [11]:
set_range

array(['200万円未満', '200~400万円未満', '400~600万円未満', '600~800万円未満',
       '800~1000万円未満', '1000~1200万円未満', '1200~1500万円未満', '1500~2000万円未満',
       '2000万円以上', 'わからない'], dtype='<U13')

In [12]:
set_number = np.arange(10) + 1

In [13]:
df_replacement = pd.DataFrame({
    'numbers': set_number,
    'ranges': set_range
})
df_replacement['numbers'] = df_replacement['numbers'].astype(int)

In [14]:
replacement_dict = dict(zip(df_replacement['numbers'], df_replacement['ranges']))

# Replace the 'income' column values in df_income using the mapping dictionary
df_income['income_replaced'] = df_income['income'].replace(replacement_dict)

In [15]:
df_income['income_replaced'].unique()

array(['400~600万円未満', '800~1000万円未満', '200万円未満', '600~800万円未満',
       '1200~1500万円未満', '200~400万円未満', 'わからない', '1500~2000万円未満',
       '1000~1200万円未満'], dtype=object)

### 収入構造の変形
2000万円以上の収入がいなかったので、`1500~2000万円未満` を `1500万円以上` に変更

In [16]:
df_filtered = df_income[df_income['income_replaced'] != 'わからない']

income_counts = df_filtered['income_replaced'].value_counts()

income_proportions = income_counts / income_counts.sum()

df_proportions = income_proportions.reset_index()
df_proportions.columns = ['ranges', 'proportion']
df_proportions = pd.merge(df_proportions, df_replacement, on='ranges')
df_proportions.loc[df_proportions['ranges'] == '1500~2000万円未満', 'ranges'] = '1500万円以上'
df_proportions = df_proportions.sort_values(by='numbers')

In [17]:
df_proportions

,ranges,proportion,numbers
3,200万円未満,0.122951,1
1,200~400万円未満,0.237705,2
0,400~600万円未満,0.262295,3
2,600~800万円未満,0.196721,4
4,800~1000万円未満,0.106557,5
7,1000~1200万円未満,0.008197,6
5,1200~1500万円未満,0.057377,7
6,1500万円以上,0.008197,8


## 日本の年収構造を取得する

In [18]:
df_standard_income = pd.read_excel(os.path.join(data_dir, 'standard_income_distribution.xlsx'), sheet_name='h20002', header=0)

In [19]:
income_range = np.array(df_standard_income.iloc[5:30, 0])

In [20]:
income_range

array(['\u3000５０万円未満', '\u3000５０～１００', '\u3000１００～１５０', '\u3000１５０～２００',
       '\u3000２００～２５０', '\u3000２５０～３００', '\u3000３００～３５０', '\u3000３５０～４００',
       '\u3000４００～４５０', '\u3000４５０～５００', '\u3000５００～５５０', '\u3000５５０～６００',
       '\u3000６００～６５０', '\u3000６５０～７００', '\u3000７００～７５０', '\u3000７５０～８００',
       '\u3000８００～８５０', '\u3000８５０～９００', '\u3000９００～９５０',
       '\u3000９５０～１０００', '\u3000１０００～１１００', '\u3000１１００～１２００',
       '\u3000１２００～１５００', '\u3000１５００～２０００', '\u3000２０００万円以上'],
      dtype=object)

In [21]:
cleaned_income_range = [clean_convert(income) for income in income_range]
cleaned_income_range = [income.replace('\u3000', '') for income in cleaned_income_range]

In [22]:
cleaned_income_range

[' 50万円未満',
 ' 50~100',
 ' 100~150',
 ' 150~200',
 ' 200~250',
 ' 250~300',
 ' 300~350',
 ' 350~400',
 ' 400~450',
 ' 450~500',
 ' 500~550',
 ' 550~600',
 ' 600~650',
 ' 650~700',
 ' 700~750',
 ' 750~800',
 ' 800~850',
 ' 850~900',
 ' 900~950',
 ' 950~1000',
 ' 1000~1100',
 ' 1100~1200',
 ' 1200~1500',
 ' 1500~2000',
 ' 2000万円以上']

In [23]:
income_distribution = np.array(df_standard_income.iloc[5:30, 1].astype(float))

In [24]:
df_stand_income_cleaned = pd.DataFrame({
    'cleaned_income_range': cleaned_income_range,
    'income_distribution': income_distribution
})

In [25]:
df_stand_income_cleaned

,cleaned_income_range,income_distribution
0,50万円未満,1.3
1,50~100,5.6
2,100~150,7.0
3,150~200,7.6
4,200~250,7.6
5,250~300,6.9
6,300~350,7.1
7,350~400,5.7
8,400~450,5.8
9,450~500,4.9


In [26]:
def aggregate_categories(df, income_cat_width):
    # Initialize the new DataFrame list
    aggregated_data = []

    # Iterate through the DataFrame in steps of income_cat_width
    for i in range(0, len(df), income_cat_width):
        # Slice the DataFrame
        slice_df = df.iloc[i:i+income_cat_width]

        # Create a new label for the aggregated category
        if slice_df['cleaned_income_range'].iloc[0].endswith('未満'):
            new_label = slice_df['cleaned_income_range'].iloc[-1].split('~')[-1] + '万円未満'
        else:
            range_start = slice_df['cleaned_income_range'].iloc[0].split('~')[0]
            range_end = slice_df['cleaned_income_range'].iloc[-1].split('~')[-1] + '万円未満'
            new_label = f'{range_start}~{range_end}'

        # Sum the income distributions for the slice
        sum_distribution = slice_df['income_distribution'].sum()

        # Append to aggregated data
        aggregated_data.append({
            'cleaned_income_range': new_label,
            'income_distribution': sum_distribution
        })

    # Convert aggregated data to DataFrame
    return pd.DataFrame(aggregated_data)

In [27]:
df_stand_income_agg_low = aggregate_categories(df_stand_income_cleaned.iloc[0:22, :], 4)
df_stand_income_high = df_stand_income_cleaned.iloc[22:, :].reset_index(drop=True)
df_high_first = pd.DataFrame([{
    'cleaned_income_range': '1200~1500万円未満',
    'income_distribution': df_stand_income_high.loc[0, 'income_distribution']
}])
df_high_second = pd.DataFrame([{
    'cleaned_income_range': '1500万円以上',
    'income_distribution': df_stand_income_high.loc[1:2, 'income_distribution'].sum()
}])

df_stand_income_agg = pd.concat([df_stand_income_agg_low, df_high_first, df_high_second])

In [28]:
df_stand_income = df_stand_income_agg
df_stand_income['income_distribution'] = df_stand_income_agg['income_distribution']/100
df_stand_income['numbers'] = np.arange(8)+1
df_stand_income['numbers'] = df_stand_income['numbers'].astype(int)

In [29]:
df_stand_income

,cleaned_income_range,income_distribution,numbers
0,200万円未満,0.215,1
1,200~400万円未満,0.273,2
2,400~600万円未満,0.192,3
3,600~800万円未満,0.121,4
4,800~1000万円未満,0.083,5
5,1000~1200万円未満,0.049,6
0,1200~1500万円未満,0.036,7
0,1500万円以上,0.032,8


## 日本の年収構造に合わせて標準化する

In [30]:
df_merged = pd.merge(df_proportions, df_stand_income_agg, on='numbers')

df_merged['weight'] = df_merged['income_distribution'] / df_merged['proportion']

df_merged = df_merged[['numbers', 'cleaned_income_range', 'proportion', 'income_distribution', 'weight']]

In [31]:
df_merged

,numbers,cleaned_income_range,proportion,income_distribution,weight
0,1,200万円未満,0.122951,0.215,1.748667
1,2,200~400万円未満,0.237705,0.273,1.148483
2,3,400~600万円未満,0.262295,0.192,0.732000
3,4,600~800万円未満,0.196721,0.121,0.615083
4,5,800~1000万円未満,0.106557,0.083,0.778923
5,6,1000~1200万円未満,0.008197,0.049,5.978000
6,7,1200~1500万円未満,0.057377,0.036,0.627429
7,8,1500万円以上,0.008197,0.032,3.904000


In [32]:
def transform_income_range(income_range):
    if income_range == '200万円未満':
        return '<200'
    elif income_range == '1500万円以上':
        return '>1500'
    else:
        return income_range.replace('万円未満', '').replace('~', ' to ')

In [33]:
df_merged_eng = df_merged.copy()
df_merged_eng['cleaned_income_range'] = df_merged_eng['cleaned_income_range'].apply(transform_income_range)

In [34]:
df_merged_eng

,numbers,cleaned_income_range,proportion,income_distribution,weight
0,1,<200,0.122951,0.215,1.748667
1,2,200 to 400,0.237705,0.273,1.148483
2,3,400 to 600,0.262295,0.192,0.732000
3,4,600 to 800,0.196721,0.121,0.615083
4,5,800 to 1000,0.106557,0.083,0.778923
5,6,1000 to 1200,0.008197,0.049,5.978000
6,7,1200 to 1500,0.057377,0.036,0.627429
7,8,>1500,0.008197,0.032,3.904000


In [35]:
df_merged_eng.to_csv(os.path.join(output_dir, 'income_distribution.csv'))

## 解析対象となった質問番号を取得する

In [36]:
df_inclusion = pd.read_excel(os.path.join(data_dir, 'ChatGPT_analysis_240401.xlsx'), sheet_name='for_analysis_240401', header=0)

In [37]:
df_overall = pd.read_excel(os.path.join(data_dir, 'ChatGPT_analysis_240401.xlsx'), sheet_name='overall_240401', header=0)

In [38]:
df_overall['new_id'] = range(1, len(df_overall) + 1)

In [39]:
df_inclusion_newid = pd.merge(df_inclusion, df_overall, on = 'No')
df_inclusion_newid['answer'] = np.where(df_inclusion_newid['No'] <= 338, 1, 3)

In [40]:
df_inclusion_newid[['No', 'new_id', 'answer']]

,No,new_id,answer
0,1,1,1
1,2,2,1
2,3,3,1
3,4,4,1
4,5,5,1
...,...,...,...
461,510,498,3
462,511,499,3
463,515,500,3
464,516,501,3


In [41]:
end_of_red = df_inclusion_newid.loc[df_inclusion_newid['answer']==1, 'new_id'].max()
end_of_red

np.int64(338)

## 解析結果をアップデートする

In [42]:
df_answer.head()

,id,SC1,SC2,SC3,SC4-1,SC4-2,SC4-3,SC4-4,SC4-5,SC4-6,...,Q502,QX1,QX2,JOB,MAR,INC,CHI,BUS,DISP,WARI
0,I/10,S/2,I/10,S/47,M/19,NaN,NaN,NaN,NaN,NaN,...,S/4,S/4,S/3,S/11,S/2,S/10,S/2,S/43,S/6,S/22
1,NaN,SC1 性別,SC2 年齢,SC3 居住地,SC4 子どもの年齢,NaN,NaN,NaN,NaN,NaN,...,Q502 ChatGPT回答に対する受診判断／子供に頭のけが・首のけが。受傷後、すでに2 日何事,QX1 ChatGPT回答に対する信頼度,QX2 ChatGPT使用による精神状態の変化,職業,未既婚,世帯年収,子供の有無,業種,DISP 回答した調査画面,WARI 設定された割付
2,NaN,NaN,NaN,NaN,0歳,1歳,2歳,3歳,4歳,5歳,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,40,14,0,0,0,0,0,0,...,NaN,3,3,3,2,3,2,19,1,6
4,2,1,52,27,0,0,0,0,0,0,...,NaN,4,2,5,2,5,1,13,1,5


In [43]:
df = df_answer.iloc[3:, [0] + list(range(24, 526))].reset_index(drop=True)
df = df.rename(columns={f"Q{i}": i for i in range(1, 503)})

start_col = 1
end_col = 502

df.loc[:, range(start_col, end_of_red + 1)] = df.loc[:, range(start_col, end_of_red + 1)].apply(
    lambda col: col.map(lambda x: 1 if x == 1 else (0 if not pd.isna(x) else np.nan))
    )

df.loc[:, range(end_of_red + 1, end_col + 1)] = df.loc[:, range(end_of_red + 1, end_col + 1)].apply(
    lambda col: col.map(lambda x: 1 if (x == 3) else (0 if not pd.isna(x) else np.nan))
)

In [44]:
columns_to_keep = ['id'] + df_inclusion_newid['new_id'].tolist()

df = df.loc[:, columns_to_keep]


In [45]:
df.head()

,id,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,502
0,1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
def get_correct_counts(df, end_of_red):

    results = pd.DataFrame([
        np.where(df.loc[:, df.columns != 'id'].sum() >= 2, 1, 0)
    ], columns=df.columns[df.columns != 'id'])

    red_columns = [col for col in results.columns if isinstance(col, int) and 1 <= col <= end_of_red]
    green_columns = [col for col in results.columns if isinstance(col, int) and end_of_red + 1 <= col <= 502]

    correct_red = (results.loc[:, red_columns] == 1).sum(axis=1).iloc[0]

    correct_green = (results.loc[:, green_columns] == 1).sum(axis=1).iloc[0]

    total_red = len(red_columns)

    total_green = len(green_columns)

    return correct_red, total_red, correct_green, total_green


In [47]:
get_correct_counts(df, end_of_red)

(np.int64(135), 314, np.int64(49), 152)

In [48]:
df_standardized = df.copy().reset_index(drop=True)
df_standardized['numbers'] = df_income['income'].astype(int)
df_standardized = pd.merge(df_standardized, df_merged_eng[['numbers', 'weight']], on = 'numbers')

columns_to_multiply = [col for col in df_standardized.columns if col not in ['id', 'numbers', 'weight']]
df_standardized[columns_to_multiply] = df_standardized[columns_to_multiply].multiply(df_standardized['weight'], axis=0)

df_base = df.copy().reset_index(drop=True)
df_base.loc[:, df_base.columns != 'id'] = df_base.loc[:, df_base.columns != 'id'].notnull().astype(int)

df_base['numbers'] = df_income['income'].astype(int)
df_base = pd.merge(df_base, df_merged_eng[['numbers', 'weight']], on = 'numbers')

df_base[columns_to_multiply] = df_base[columns_to_multiply].multiply(df_base['weight'], axis=0)


In [49]:
df_standardized

,id,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,500,501,502,numbers,weight
0,1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0.732000
1,2,NaN,NaN,NaN,NaN,0.778923,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.778923
2,3,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.748667
3,4,NaN,0.615083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0.615083
4,5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.778923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,1.148483
118,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,3,0.732000
119,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.627429,NaN,NaN,NaN,7,0.627429
120,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,5,0.778923


In [50]:
df_base

,id,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,500,501,502,numbers,weight
0,1,0.0,0.0,0.0,0.0,0.0,0.732,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.732000
1,2,0.0,0.0,0.0,0.0,0.778923,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.778923
2,3,0.0,0.0,0.0,0.0,0.0,1.748667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.748667
3,4,0.0,0.615083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.615083
4,5,0.0,0.0,0.778923,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.778923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.148483,0.0,0.0,0.0,2,1.148483
118,195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.732,0.0,0.0,0.0,0.0,0.0,3,0.732000
119,197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.627429,0.0,0.0,0.0,7,0.627429
120,198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.778923,0.0,0.0,0.0,0.0,5,0.778923


In [51]:
red_columns = [col for col in df.columns if isinstance(col, int) and 1 <= col <= end_of_red]
green_columns = [col for col in df.columns if isinstance(col, int) and end_of_red + 1 <= col <= 502]

In [52]:
df_standardized_red = df_standardized[red_columns]
df_standardized_green = df_standardized[green_columns]
df_base_red = df_base[red_columns]
df_base_green = df_base[green_columns]

In [53]:
correct_red = df_standardized_red.sum()
base_red = df_base_red.sum()
np.sum(correct_red >= base_red*2/3)

np.int64(105)

In [54]:
correct_green = df_standardized_green.sum()
base_green = df_base_green.sum()
np.sum(correct_green >= base_green*2/3)

np.int64(42)

In [55]:
exclude_id = df_income.loc[df_income['income'] == 10, 'id']
df_not_excluded = df.loc[~df['id'].isin(exclude_id), :]

In [56]:
np.sum(df_not_excluded.loc[:, df_not_excluded.columns != 'id'].sum())

481.0

In [57]:
np.sum(df_standardized[columns_to_multiply].sum())

478.6329923266402